In [ ]:
import xlwings as xw
import pandas as pd
import os
import time


# === Paramètres ===
fichier_source = r"C:\Users\admin\Desktop\ETL_Budget_Automation\ETL_Budget_Automation\data\Template_PMP_Projet_Test_Proximus_KPN.xlsx"  # Le fichier multi-feuilles
nom_feuille = "Quoi tester"  # La feuille à traiter
fichier_sortie = r"C:\Users\admin\Desktop\ETL_Budget_Automation\ETL_Budget_Automation\outputs\extraction.xlsx"
decalage_lignes = -4  # Pour corriger le décalage des cases à cocher

# Lancer l'application Excel
app = xw.App(visible=False)

try:
    wb = app.books.open(fichier_source)
    
    # Vérifie que la feuille existe
    if nom_feuille not in [s.name for s in wb.sheets]:
        raise ValueError(f"La feuille '{nom_feuille}' n'existe pas dans le fichier.")

    sheet = wb.sheets[nom_feuille]

    # Lire les données
    df = sheet.used_range.options(pd.DataFrame, header=1, index=False).value
    df = df.dropna(how='all').reset_index(drop=True)

    # Corriger la première colonne
    col0 = str(df.columns[0]).strip().lower()
    if col0 in ["none", "nan", "na"]:
        df = df.rename(columns={df.columns[0]: "Famille Technique"})
    df.iloc[:, 0] = df.iloc[:, 0].ffill()

    # Ajouter les colonnes à remplir
    df["Demande ICP"] = "Non"
    df["Soft de Test"] = "Non"

    # Récupérer toutes les cases à cocher
    checkboxes = sheet.api.CheckBoxes()

    print(f"\n✅ Total cases détectées : {checkboxes.Count}")
    for i in range(1, checkboxes.Count + 1):
        cb = checkboxes.Item(i)
        row = cb.TopLeftCell.Row
        col = cb.TopLeftCell.Column
        val = cb.Value

        # Appliquer le décalage pour mapper sur le DataFrame
        ligne_df = row + decalage_lignes - 1  # -1 car index Python

        if ligne_df < 0 or ligne_df >= len(df):
            continue  # Ignorer les lignes hors limites

        # Affecter les valeurs aux bonnes colonnes selon la colonne d'origine
        if col == 6:  # Colonne F
            df.at[ligne_df, "Demande ICP"] = "Oui" if val == 1 else "Non"
        elif col == 7:  # Colonne G
            df.at[ligne_df, "Soft de Test"] = "Oui" if val == 1 else "Non"

finally:
    time.sleep(2)
    wb.close()
    app.quit()



✅ Total cases détectées : 619


In [6]:
df.head(50)

,Famille Technique,Fonctions a tester,Demande ICP,Soft de test,Commentaire,Soft de Test
0,TF,USB 3.0,Non,None,None,Non
1,TF,USB 2.0,Non,None,None,Non
2,TF,Consomation,Oui,None,None,Oui
3,TF,Trans 300 MHRZ,Oui,None,None,Oui
4,TF,Trans 1000 MHRZ,Oui,None,None,Oui
5,TF,Trans 3400 MHRZ,Oui,None,None,Oui
6,TF,Tension de sonnerie,Oui,None,None,Oui
7,TF,Tension de repos,Oui,None,None,Oui
8,TF,FXS1,Oui,None,None,Oui
9,TF,FXS2,Oui,None,None,Oui


In [7]:
# === Nettoyage des colonnes vides ===
colonnes_vides = df.columns[df.isnull().all()]
df.drop(columns=colonnes_vides, inplace=True)
print(f"\n🧹 Colonnes supprimées (entièrement vides) : {list(colonnes_vides)}")

# === Remplissage des colonnes partiellement vides ===
# Exemple : remplir les NaN de la colonne "Commentaires"
if "Commentaire" in df.columns:
    df["Commentaire"] = df["Commentaire"].fillna("Pas de commentaire")
    print("📝 NaN remplacés par 'Pas de commentaire' dans la colonne 'Commentaires'.")



🧹 Colonnes supprimées (entièrement vides) : ['Soft de test ']
📝 NaN remplacés par 'Pas de commentaire' dans la colonne 'Commentaires'.


In [8]:
df.head()

,Famille Technique,Fonctions a tester,Demande ICP,Commentaire,Soft de Test
0,TF,USB 3.0,Non,Pas de commentaire,Non
1,TF,USB 2.0,Non,Pas de commentaire,Non
2,TF,Consomation,Oui,Pas de commentaire,Oui
3,TF,Trans 300 MHRZ,Oui,Pas de commentaire,Oui
4,TF,Trans 1000 MHRZ,Oui,Pas de commentaire,Oui


In [9]:
# Sauvegarde finale
if os.path.exists(fichier_sortie):
        os.remove(fichier_sortie)
df.to_excel(fichier_sortie, index=False)
print(f"\n📁 Fichier sauvegardé ici : {fichier_sortie}")


📁 Fichier sauvegardé ici : C:\Users\imen\Desktop\ETL_Budget_Automation\outputs\extraction.xlsx
